<center>

<div>
<img src="images/statdatahub.png" width="200"/>
</div>

# Explorando el Lenguaje: Introducción a la Minería de Textos y NLP

<h2>Introducción</h2>
<p><strong>Contexto del problema</strong>. En la actualidad, los relatos y narrativas sobre los desafíos culturales que enfrenta Colombia son una fuente invaluable de información para analizar los retos que enfrenta el país en esta área. Estos textos, que van desde opiniones de expertos hasta comentarios ciudadanos, reflejan una diversidad de perspectivas y matices que pueden ser analizados mediante técnicas de procesamiento de lenguaje natural (NLP). Este ejercicio busca aplicar herramientas como Bag of Words (BOW), One-Hot Encoding y el cálculo de la frecuencia inversa de documentos (TfIDF) para extraer información clave, identificar patrones comunes y entender las principales preocupaciones y retos relacionados con la cultura en Colombia. El análisis de texto nos permitirá clasificar, interpretar y agrupar estos relatos para obtener una visión más clara sobre los retos culturales más urgentes.</p>

<p><strong>Contexto analítico</strong>. El archivo de datos "retos_culturales_colombia.csv" contiene textos narrativos que describen los desafíos culturales de Colombia. Cada fila corresponde a un relato, y las columnas contienen diversas características del texto. Para el análisis, se aplicarán técnicas como Bag of Words, que convierte los relatos en matrices de frecuencia de palabras; One-Hot Encoding, que representa las palabras como vectores binarios; y TfIDF, que pondera la importancia de cada palabra en función de su frecuencia en el corpus. Estas técnicas permitirán detectar las palabras más relevantes y establecer relaciones entre los diferentes relatos, facilitando así la identificación de temas recurrentes y patrones ocultos en los datos textuales.</p>

# Datos

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk # imports the natural language toolkit
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

In [ ]:
# Cargar Conjunto de datos
data = pd.read_excel('data/retos_culturales_colombia.xlsx')
data.head()

# Preprocesamiento de textos

- **Para qué?:** Para eliminar signos de puntuación y palabras conectoras (stopwords) que no nos aportan al análisis del texto.
- **Por qué?:** Obtenemos un vocabulario más limpio, lo que nos permitirá generar resultados más confiables

In [ ]:
AllReviews = data['Reto']
AllReviews.head()

In [ ]:
n=len(AllReviews)
n

In [ ]:
AllReviews=[AllReviews[index].lower() for index in range(n)]

In [ ]:
# Quitar puntuaciones
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
AllReviews2 = AllReviews.copy()

for op in range(n):
    my_str = AllReviews[op]
    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
       if char not in punctuations:
           no_punct = no_punct + char

    # display the unpunctuated string
    AllReviews2[op]=no_punct

# Eliminación de stopwords

- **Para qué:** Eliminar palabras comunes que probablemente aparecerán en cualquier texto
- **Por qué:** Ellas no te dicen mucho sobre tu texto

In [ ]:
# Quitar stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words("spanish"))
AllReviews3 = AllReviews2.copy()

for op in range(n):
    without_stop_words = []
    stopword = []
    sentence = AllReviews2[op]
    words = nltk.word_tokenize(sentence)
    for word in words:
        if word in stop_words:
            stopword.append(word)
        else:
            without_stop_words.append(word)
    AllReviews3[op]= ' '.join(without_stop_words)

# Derivación y lematización

**Derivación:**

- **Para qué:** Reducir una palabra a su forma base/raíz
- **Por qué:** A menudo tiene sentido tratar las palabras relacionadas de la misma manera.
- **Comentarios:**
    - Utiliza un enfoque basado en reglas "simple" y rápido
    - Las palabras derivadas generalmente no se muestran a los usuarios (se utilizan para análisis/indexación)
    - Algunos motores de búsqueda tratan las palabras con la misma raíz como sinónimos.

In [ ]:
# initialize stemmer
stemmer = SnowballStemmer('spanish')

In [ ]:
vect = CountVectorizer()
vect.fit(AllReviews3)

In [ ]:
words = list(vect.vocabulary_.keys())[:100]

In [ ]:
# stem each word
print([stemmer.stem(word) for word in words])

**Lematización**

- **Para qué:** Derivar la forma canónica ('lema') de una palabra
- **Por qué:** Puede ser mejor que el uso de la derivación
- **Comentario:** Utiliza un enfoque basado en diccionario (más lento que la derivación)

In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
# assume every word is a noun
print([wordnet_lemmatizer.lemmatize(word) for word in words])

In [ ]:
# assume every word is a verb
print([wordnet_lemmatizer.lemmatize(word,pos='v') for word in words])

In [ ]:
# define a function that accepts text and returns a list of lemmas
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

In [ ]:
vect = CountVectorizer(analyzer=split_into_lemmas)

# Tokenización

- **Para qué?:** Separar el texto en unidades como oraciones o palabras.
- **Por qué?:** Da estructura a un texto previamente no estructurado
- **Comentario:** Relativamente fácil con textos en inglés, no fácil con algunos idiomas.

In [ ]:
# use CountVectorizer to create document-term matrices from X
X_dtm = vect.fit_transform(AllReviews3)

In [ ]:
X_dtm

In [ ]:
X_dtm.todense()

In [ ]:
temp=X_dtm.todense()

In [ ]:
import itertools 

out = dict(itertools.islice(vect.vocabulary_.items(), 5)) 

str(out)

In [ ]:
# rows are documents, columns are terms (aka "tokens" or "features")
X_dtm.shape

# n-gramas

In [ ]:
from nltk.util import ngrams

def top_k_ngrams(word_tokens,n,k):
    
    ## Getting them as n-grams
    n_gram_list = list(ngrams(word_tokens, n))

    ### Getting each n-gram as a separate string
    n_gram_strings = [' '.join(each) for each in n_gram_list]
    
    n_gram_counter = Counter(n_gram_strings)
    most_common_k = n_gram_counter.most_common(k)
    return(most_common_k)

In [ ]:
from collections import Counter

all_reviews_text = ' '.join(AllReviews3)
tokenized_words = nltk.word_tokenize(all_reviews_text)

tri_gramas=top_k_ngrams(tokenized_words, 3, 30)

In [ ]:
tri_gramas

#  Term Frequency-Inverse Document Frequency (TF-IDF)

- **Para qué:** Calcula la "frecuencia relativa" con la que aparece una palabra en un documento en comparación con su frecuencia en todos los documentos.
- **Por qué:** Más útil que "frecuencia de términos" para identificar palabras "importantes" en cada documento (alta frecuencia en ese documento, baja frecuencia en otros documentos)
- **Comentario:** Se utiliza para puntuación de motores de búsqueda, resumen de texto y agrupamiento de documentos.

In [ ]:
# create a document-term matrix using TF-IDF
vect = TfidfVectorizer()
dtm = vect.fit_transform(AllReviews3)
X = pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())
features = vect.vocabulary_.keys()
dtm.shape

In [ ]:
# choose a random text
review_id = 40
review_text = AllReviews3[review_id]
review_length = len(review_text)

In [ ]:
# create a dictionary of words and their TF-IDF scores
word_scores = {}
for word in vect.vocabulary_.keys():
    word = word.lower()
    if word in features:
        word_scores[word] = dtm[review_id, list(features).index(word)]

In [ ]:
# print words with the top 5 TF-IDF scores
print('TOP SCORING WORDS:')
top_scores = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[:5]
for word, score in top_scores:
    print(word)

In [ ]:
from sklearn.cluster import KMeans
# Se eligen 5 grupos por el método 
true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=1)
model.fit(X)

In [ ]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vect.get_feature_names()

In [ ]:
for i in range(true_k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :20]:
        print(' %s' % terms[ind])

In [ ]:
#pip install threadpoolctl==3.1.0

In [ ]:
print("Prediction")
predicted = model.predict(X)
print(predicted)

In [ ]:
data['Cluster'] = predicted

In [ ]:
list(data.query("Cluster==0").iloc[:10]['Reto'])

In [ ]:
list(data.query("Cluster==1").iloc[:10]['Reto'])